<a href="https://colab.research.google.com/github/shalinisaini/CS60092_9A_18CS60R47/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
SEED =42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


In [147]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv'
df1 = pd.read_csv(url, sep='\t')

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df2 = pd.read_csv(url, sep='\t')

In [0]:
train_data = df2['text'].astype(str).tolist()
train_label = df2['label'].astype(int)
test_data = df1['text'].astype(str).tolist()
test_label = df1['label'].astype(int)

train_data = [x.lower() for x in train_data]
test_data = [x.lower() for x in test_data]


 


In [0]:
def preprocess(data):
  stopword = set(stopwords.words('english'))
  dt2=[]
  for i in data:
    words = i.split(" ")
    dt =[]
    for j in words:
      if(j not in stopword and not j.isdigit() and len(j)>1):
        dt.append(j)
    dt2.append(dt)
  return dt2

              

In [0]:

train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [153]:
print(len(train_data))

17999


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  print(dataset)
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = 1
          else:
              word_to_ix[word] = word_to_ix[word]+1
  return word_to_ix

In [155]:
word_to_ix = generate_word_ids(train_data)
print(len(word_to_ix))
NUM_LABELS = 2



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



204915


In [156]:
list=[]
for i in word_to_ix:
  if(word_to_ix[i]>10):
    list.append(i)
word_to_ix={}
for i in list:
  word_to_ix[i] = len(word_to_ix)
  
VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)

19704


In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 100)
    #self.lin2 = nn.Linear(50, 50)
    #self.lin3 = nn.Linear(50, 50)
    #self.lin4 = nn.Linear(100, 100)
    #self.lin5 = nn.Linear(100, 100)
    #self.lin6 = nn.Linear(200, 200)
    #self.lin7 = nn.Linear(200, 200)
    #self.lin8 = nn.Linear(500, 500)
    #self.lin9 = nn.Linear(500, 500)
    #self.lin10 = nn.Linear(10, 10)
    #self.lin11= nn.Linear(10, 10)
    #self.lin12= nn.Linear(10, 10)
    #self.lin13 = nn.Linear(20, 20)
    #self.lin14= nn.Linear(20, 10)
    #self.lin15= nn.Linear(10, 10)
    #self.lin16 = nn.Linear(30, 30)
    #self.lin17= nn.Linear(30, 30)
    #self.lin18= nn.Linear(30, 30)
    #self.lin19 = nn.Linear(50, 50)
    #self.lin20= nn.Linear(50, 50)
    #self.lin21= nn.Linear(50, 50)
    #self.lin22 = nn.Linear(200, 200)
    #self.lin23= nn.Linear(500, 500)
    #self.lin24= nn.Linear(500, 500)
    self.lin4 = nn.Linear(100,50)
    self.lin5 = nn.Linear(50,10)
    self.lin6 = nn.Linear(10, num_labels)
    
    
    
  def forward(self, x):
    x=self.lin4(x)
    x=self.dp(x)
    return F.softmax(self.lin6(self.lin5(self.lin4(self.lin(x)))))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            c=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()


In [0]:
def accuracy(data,label):
  count=0
  for idx in range(len(data)):
      bow_vec = Variable(make_bow_vector(data[idx], word_to_ix)).cuda()
      labels = Variable(torch.LongTensor(label[idx])).cuda()
      logprobs = bow(bow_vec).reshape(-1)
      val,pred = logprobs.max(0)
      #print(pred,label[idx]+1)
      if(pred == label[idx]):
        count+=1
  print("accuracy",count/len(data))
        


In [0]:
# define a loss function and an optimizer
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [162]:
# the training loop
for epoch in range(5):
    print("epoch",epoch)
    for idx in range(len(train_data)):
        # get the training data
        bow.zero_grad()
        #print(train_data[idx])
        #print(train_label[idx])
        bow_vec = Variable(make_bow_vector(train_data[idx], word_to_ix)).cuda()
        label  = Variable(torch.LongTensor([train_label[idx]])).cuda()
        #print(bow_vec,label,train_label[idx])
        probs = bow(bow_vec) # forward pass
        #print(probs)
        loss = loss_function(probs, label)
        loss.backward()
        opt.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    accuracy(train_data,train_label)


epoch 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


CURRENT LOSS: 0.3491954207420349
accuracy 0.8804378021001167
epoch 1
CURRENT LOSS: 0.3133314847946167
accuracy 0.8683815767542641
epoch 2
CURRENT LOSS: 0.31326162815093994
accuracy 0.863492416245347
epoch 3
CURRENT LOSS: 0.31326162815093994
accuracy 0.916439802211234
epoch 4
CURRENT LOSS: 0.31326186656951904
accuracy 0.9213289627201511


In [163]:
accuracy(test_data,test_label)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


accuracy 0.8536


In [164]:
torch.save(bow.cpu(),'Task3A.mdl')
from google.colab import files
files.download("Task3A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
